In [ ]:
#RESNET
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, Add,
GlobalAveragePooling2D, Dense, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
# Load the SVHN dataset
(ds_train, ds_test), ds_info = tfds.load(
'svhn_cropped',
split=['train', 'test'],
shuffle_files=True,
as_supervised=True,
with_info=True,
)
# Preprocess the data
def preprocess(image, label):
    image = tf.cast(image, tf.float32) / 255.0
label = tf.one_hot(label, depth=10)
return image, label
AUTOTUNE = tf.data.AUTOTUNE
train_ds = ds_train.map(preprocess, num_parallel_calls=AUTOTUNE)
test_ds = ds_test.map(preprocess, num_parallel_calls=AUTOTUNE)
# Batch and prefetch the data
BATCH_SIZE = 32
train_ds = train_ds.cache().shuffle(1000).batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.batch(BATCH_SIZE).cache().prefetch(buffer_size=AUTOTUNE)
# Define ResNet block
def resnet_block(inputs, filters, kernel_size=3, stride=1):
x = Conv2D(filters, kernel_size=kernel_size, strides=stride, padding='same')(inputs)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Conv2D(filters, kernel_size=kernel_size, strides=1, padding='same')(x)
x = BatchNormalization()(x)
if stride != 1 or inputs.shape[-1] != filters:
inputs = Conv2D(filters, kernel_size=1, strides=stride, padding='same')(inputs)
inputs = BatchNormalization()(inputs)
x = Activation('relu')(x)
return x
# Build ResNet model
def build_resnet(input_shape, num_classes):
    inputs = Input(shape=input_shape)
x = Conv2D(32, kernel_size=3, strides=1, padding='same')(inputs)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = resnet_block(x, 32)
x = resnet_block(x, 32)
x = resnet_block(x, 64, stride=2)
x = resnet_block(x, 64)
x = resnet_block(x, 128, stride=2)
x = resnet_block(x, 128)
x = GlobalAveragePooling2D()(x)
x = Flatten()(x)
outputs = Dense(num_classes, activation='softmax')(x)
model = Model(inputs, outputs)
return model
# Build and compile the model
input_shape = (32, 32, 3)
num_classes = 10
model = build_resnet(input_shape, num_classes)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Print model summary
model.summary()
# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
width_shift_range=0.1,
height_shift_range=0.1,
horizontal_flip=True
)
EPOCHS = 5
# Convert the dataset to NumPy arrays
train_images = []
train_labels = []
for image, label in train_ds:
    train_images.append(image.numpy())
train_labels.append(label.numpy())
train_images = np.concatenate(train_images, axis=0) # Concatenate the image tensors along
    the first axis
train_labels = np.concatenate(train_labels, axis=0)
history = model.fit(datagen.flow(train_images,
train_labels,
batch_size=BATCH_SIZE),
validation_data=test_ds,
epochs=EPOCHS)
loss, accuracy = model.evaluate(test_ds)
print(f"Test Accuracy: {accuracy * 100:.2f}%")